## The Core Intuition: System 1 and System 2

Imagine how you play a game. You have two distinct ways of thinking (popularized by Daniel Kahneman):

**1. System 1 (Fast, Intuitive)**
You look at a face and instantly recognize it. In AlphaZero, this is the **Neural Network**. It looks at the board and instantly feels: *"Move here"* or *"I'm winning"*. It's fast, creative, but sometimes wrong.

**2. System 2 (Slow, Deliberate)**
You solve a math problem or calculate chess variations. *"If I go there, he goes there..."* In AlphaZero, this is **Monte Carlo Tree Search (MCTS)**. It simulates the future to verify if the intuition is correct.

**The AlphaZero Magic**
Traditional engines (like Stockfish) rely on massive calculation (System 2). Traditional AI relies on pattern matching (System 1). AlphaZero combines them:
*   The **Network** guides the Search (telling it where to look).
*   The **Search** improves the Network (generating data to correct its blind spots).

```mermaid
graph LR
    subgraph System 1: Intuition
    A[Board State] -->|Neural Network| B(Policy: Move Probabilities)
    A -->|Neural Network| C(Value: Win Probability)
    end
    
    subgraph System 2: Reasoning
    B --> D[MCTS Search]
    C --> D
    D --> E[Final Move Selection]
    end
    
    style A fill:#f9f,stroke:#333,stroke-width:2px
    style E fill:#ccf,stroke:#333,stroke-width:2px
```

## The "Chicken and Egg" Problem

To build this, we face a paradox:
1.  To play well, we need **Search** (MCTS).
2.  But Search needs a good **Network** to guide it (otherwise it's just random guessing).
3.  But the Network needs **Training Data** to learn what good moves look like.
4.  But to get Training Data, we need an agent that plays well!

**The Solution: The Virtuous Cycle**
We start with a completely random, "dumb" agent.
1.  **Self-Play:** Even a random Search is slightly better than a raw guess because it sees a few steps ahead. We let the agent play thousands of games against itself.
2.  **Training:** We use those games to train the Network.
3.  **Improvement:** The Network becomes slightly less dumb. It now guides the Search better.
4.  **Repeat:** We run the cycle again. The agent pulls itself up by its bootstraps, eventually becoming superhuman.



```mermaid
---
config:
  theme: 'base'
  themeVariables:
    primaryColor: '#ffffff'
    primaryTextColor: '#4D5461'
    primaryBorderColor: '#096bda11'
    lineColor: '#096bda'
    secondaryColor: '#ffffff84'
    secondaryTextColor: '#4D5461'
    tertiaryColor: '#4d546110'
    tertiaryTextColor: '#4D5461'
---
flowchart TD
    A[Self-Play] -->|Generates Games| B(Game Data)
    B -->|Used to| C(Train Neural Network)
    C -->|Improved Intuition| D(Better MCTS Search)
    D -->|Higher Quality Play| A
    
    style A fill:#bfb,stroke:#333,stroke-width:2px
    style C fill:#fbf,stroke:#333,stroke-width:2px
```

## Learning Path

We've broken this complex system into three digestible parts. It is best to read them in order:
1. [αMCTS](mcts.html)
2. [Neural Network](nn.html)
3. [Self-Play](self-play.html)

### [αMCTS](mcts.html)
How we build a search tree that balances "trying new things" (exploration) vs "sticking to what works" (exploitation).

In [6]:
svg_str_mcts = """
<svg xmlns="http://www.w3.org/2000/svg" viewBox="20 0 260 260" width="100%" height="260" role="img" aria-hidden="true">
  <style>
    /* Prefer theme CSS custom properties, with fallbacks to original values */
    :root{
      --accent-svg: var(--accent, #0969da);
      --stroke-svg: var(--accent, #0969da);
      --muted-svg: var(--text, #1D1D20);
    }
    svg{font-family:Arial,Helvetica,sans-serif}
    .draw{stroke-dasharray:300;stroke-dashoffset:300;animation:draw 900ms cubic-bezier(.2,.8,.2,1) forwards;fill:none;stroke-linecap:round;stroke-linejoin:round}
    .level1 .draw { animation-delay: 0.5s; }
    .level2 .draw { animation-delay: 3.0s; }
    .level3 .draw { animation-delay: 4.5s; }
    @keyframes draw{to{stroke-dashoffset:0}}
    .node{transform-box:fill-box;transform-origin:center;opacity:0;animation:appear 0.3s ease forwards;fill:#fff;stroke:var(--muted-svg);stroke-width:1.6;stroke-linecap:round;stroke-linejoin:round}
    .level0 .node { opacity:1;  }
    .level1 .node { animation-delay: 0.8s, 0.8s; }
    .level2 .node { animation-delay: 2.3s, 2.3s; }
    .level3 .node { animation-delay: 3.3s, 3.3s; }
    @keyframes appear { from { opacity: 0; } to { opacity: 1; } }
  </style>
  <g transform="translate(40,20)">
    <!-- level 0 -->
    <g class="level0">
      <!-- root node rendered later to be on top -->
    </g>
    <!-- level 1 (2 children) -->
    <g class="level1">
      <!-- connections -->
      <path class="draw" stroke="var(--stroke-svg)" stroke-width="2.4" d="M94 8 C78 40 60 56 44 76"/>
      <path class="draw" stroke="var(--stroke-svg)" stroke-width="2.4" d="M94 8 C110 40 130 56 148 76"/>
    </g>
    <!-- level 2 (each child expands to 3 children) -->
    <g class="level2">
      <!-- left subtree edges -->
      <path class="draw" stroke="var(--stroke-svg)" stroke-width="2" d="M44 76 C34 100 26 118 24 138"/>
      <path class="draw" stroke="var(--stroke-svg)" stroke-width="2" d="M44 76 C52 102 60 120 74 138"/>
      <path class="draw" stroke="var(--stroke-svg)" stroke-width="2" d="M44 76 C10 108 6 142 12 168"/>
      <!-- right subtree edges -->
      <path class="draw" stroke="var(--stroke-svg)" stroke-width="2" d="M148 76 C138 100 126 118 120 138"/>
      <path class="draw" stroke="var(--stroke-svg)" stroke-width="2" d="M148 76 C156 102 170 120 186 138"/>
      <path class="draw" stroke="var(--stroke-svg)" stroke-width="2" d="M148 76 C198 110 212 142 204 172"/>
    </g>
    <!-- level 3 (deeper branching on middle branches) -->
    <g class="level3">
      <!-- connect one of the left-level nodes to two more -->
      <path class="draw" stroke="var(--stroke-svg)" stroke-width="1.8" d="M74 138 C86 158 102 170 116 188"/>
      <path class="draw" stroke="var(--stroke-svg)" stroke-width="1.8" d="M74 138 C64 166 56 184 42 200"/>
    </g>
    <!-- nodes (rendered on top) -->
    <g class="level0_nodes">
      <circle class="node" cx="94" cy="8" r="6.8" fill="#fff" stroke="var(--accent-svg)" stroke-width="2"/>
    </g>
    <g class="level1_nodes">
      <circle class="node" cx="44" cy="76" r="5.2" fill="#fff" stroke="var(--muted-svg)" stroke-width="1.6"/>
      <circle class="node" cx="148" cy="76" r="5.2" fill="#fff" stroke="var(--muted-svg)" stroke-width="1.6"/>
    </g>
    <g class="level2_nodes">
      <circle class="node" cx="24" cy="138" r="4.8" fill="#fff" stroke="var(--muted-svg)" stroke-width="1.4"/>
      <circle class="node" cx="74" cy="138" r="4.8" fill="#fff" stroke="var(--muted-svg)" stroke-width="1.4"/>
      <circle class="node" cx="12" cy="168" r="4.8" fill="#fff" stroke="var(--muted-svg)" stroke-width="1.4"/>
      <circle class="node" cx="120" cy="138" r="4.8" fill="#fff" stroke="var(--muted-svg)" stroke-width="1.4"/>
      <circle class="node" cx="186" cy="138" r="4.8" fill="#fff" stroke="var(--muted-svg)" stroke-width="1.4"/>
      <circle class="node" cx="204" cy="172" r="4.8" fill="#fff" stroke="var(--muted-svg)" stroke-width="1.4"/>
    </g>
    <g class="level3_nodes">
      <circle class="node" cx="116" cy="188" r="4.4" fill="#fff" stroke="var(--muted-svg)" stroke-width="1.2"/>
      <circle class="node" cx="42" cy="200" r="4.4" fill="#fff" stroke="var(--muted-svg)" stroke-width="1.2"/>
    </g>
  </g>
</svg>
"""

# Display the MCTS SVG
from IPython.display import HTML, display

display(HTML(svg_str_mcts))

### NN
How we encode the board and interpret the two "Heads" (Policy and Value) to guide our search.

In [7]:
svg_str_nn = """
<svg xmlns="http://www.w3.org/2000/svg" viewBox="30 40 240 160" width="100%" height="160" role="img" aria-hidden="true">
  <style>
    /* Prefer global theme variables when available, otherwise fall back */
    :root{
      --accent-svg: var(--accent, #0969da);
      --conn-svg: var(--accent, var(--accent, #0969da));
      --nodeStroke-svg: var(--nodeStroke, #24333f);
    }
    /* Nodes: appear staggered per-layer and row, then pulse */
    .node{fill:#fff;stroke:var(--nodeStroke-svg);stroke-width:1.6;transform-box:fill-box;transform-origin:center;opacity:0;}
    .node.visible{opacity:1}
    @keyframes appear { from { opacity: 0; transform:translateY(6px) scale(0.98)} to { opacity: 1; transform:translateY(0) scale(1)} }
    @keyframes pulse{0%{transform:scale(1)}50%{transform:scale(1.08)}100%{transform:scale(1)}}

    /* connection lines: visible baseline + animated overlay */
    /* baseline connections — always visible (faint) */
    .conn{stroke:var(--conn-svg);stroke-width:1.2;stroke-linecap:round;fill:none;stroke-dasharray:0;stroke-dashoffset:0;opacity:1;stroke-opacity:.45}
    /* animated overlay — draws on top when supported */
    .conn-anim{stroke:var(--conn-svg);stroke-width:1.2;stroke-linecap:round;fill:none;stroke-dasharray:260;stroke-dashoffset:260;stroke-opacity:0.75}
    @keyframes drawDash { to { stroke-dashoffset:0; stroke-opacity:1 } }
    .conn-anim.fast{animation:drawDash 700ms ease forwards}
    .conn-anim.slow{animation:drawDash 1100ms ease forwards}

    /* helper classes for stagger timings */
    .layer-left .node{animation:appear 360ms ease forwards}
    .layer-middle .node{animation:appear 360ms ease forwards;}
    .layer-right .node{animation:appear 360ms ease forwards;}

    /* per-row delays for a subtle wave */
    .r1{animation-delay:0.15s}.r2{animation-delay:0.30s}.r3{animation-delay:0.45s}
    .r4{animation-delay:0.60s}.r5{animation-delay:0.75s}

    /* middle/right layers will start after connections draw */
    .layer-middle .node{animation-delay:calc(var(--row-delay,0s) + 0.9s)}
    .layer-right .node{animation-delay:calc(var(--row-delay,0s) + 1.9s)}

    /* small pulse applied after appear completes */
    .node.pulse{animation: pulse 1600ms ease-in-out infinite; animation-delay: 2s}
  </style>

  <g transform="translate(24,36)">
    <!-- connections left->mid (drawn after left nodes appear) -->
    <g class="conn conn-left-mid">
      <path class="conn c c1 fast" d="M16 20 Q 58 20 124 20" style="animation-delay:0.85s"/>
      <path class="conn c c2 fast" d="M16 20 Q 82 30 124 52" style="animation-delay:0.88s"/>
      <path class="conn c c3 fast" d="M16 20 Q 62 45 124 84" style="animation-delay:0.91s"/>
      <path class="conn c c4 fast" d="M16 20 Q 78 60 124 116" style="animation-delay:0.94s"/>
      <path class="conn c c5 fast" d="M16 20 Q 54 75 124 148" style="animation-delay:0.97s"/>
      <path class="conn c c6 fast" d="M16 52 Q 72 38 124 20" style="animation-delay:1.00s"/>
      <path class="conn c c7 fast" d="M16 52 Q 66 52 124 52" style="animation-delay:1.03s"/>
      <path class="conn c c8 fast" d="M16 52 Q 76 65 124 84" style="animation-delay:1.06s"/>
      <path class="conn c c9 fast" d="M16 52 Q 60 80 124 116" style="animation-delay:1.09s"/>
      <path class="conn c c10 fast" d="M16 52 Q 80 95 124 148" style="animation-delay:1.12s"/>
      <path class="conn c c11 fast" d="M16 84 Q 56 55 124 20" style="animation-delay:1.15s"/>
      <path class="conn c c12 fast" d="M16 84 Q 84 68 124 52" style="animation-delay:1.18s"/>
      <path class="conn c c13 fast" d="M16 84 Q 68 84 124 84" style="animation-delay:1.21s"/>
      <path class="conn c c14 fast" d="M16 84 Q 74 98 124 116" style="animation-delay:1.24s"/>
      <path class="conn c c15 fast" d="M16 84 Q 52 112 124 148" style="animation-delay:1.27s"/>
      <path class="conn c c16 fast" d="M16 116 Q 88 72 124 20" style="animation-delay:1.30s"/>
      <path class="conn c c17 fast" d="M16 116 Q 64 85 124 52" style="animation-delay:1.33s"/>
      <path class="conn c c18 fast" d="M16 116 Q 76 98 124 84" style="animation-delay:1.36s"/>
      <path class="conn c c19 fast" d="M16 116 Q 58 116 124 116" style="animation-delay:1.39s"/>
      <path class="conn c c20 fast" d="M16 116 Q 86 130 124 148" style="animation-delay:1.42s"/>
      <path class="conn c c21 fast" d="M16 148 Q 60 88 124 20" style="animation-delay:1.45s"/>
      <path class="conn c c22 fast" d="M16 148 Q 82 102 124 52" style="animation-delay:1.48s"/>
      <path class="conn c c23 fast" d="M16 148 Q 62 114 124 84" style="animation-delay:1.51s"/>
      <path class="conn c c24 fast" d="M16 148 Q 78 130 124 116" style="animation-delay:1.54s"/>
      <path class="conn c c25 fast" d="M16 148 Q 54 148 124 148" style="animation-delay:1.57s"/>
    </g>

    <!-- connections mid->right (drawn after middle nodes appear) -->
    <g class="conn conn-mid-right slow">
      <path class="conn m m1 slow" d="M124 20 Q 170 20 232 20" style="animation-delay:2.75s"/>
      <path class="conn m m2 slow" d="M124 20 Q 190 30 232 52" style="animation-delay:2.78s"/>
      <path class="conn m m3 slow" d="M124 20 Q 166 45 232 84" style="animation-delay:2.81s"/>
      <path class="conn m m4 slow" d="M124 20 Q 194 60 232 116" style="animation-delay:2.84s"/>
      <path class="conn m m5 slow" d="M124 20 Q 172 75 232 148" style="animation-delay:2.87s"/>
      <path class="conn m m6 slow" d="M124 52 Q 188 38 232 20" style="animation-delay:2.90s"/>
      <path class="conn m m7 slow" d="M124 52 Q 164 52 232 52" style="animation-delay:2.93s"/>
      <path class="conn m m8 slow" d="M124 52 Q 196 65 232 84" style="animation-delay:2.96s"/>
      <path class="conn m m9 slow" d="M124 52 Q 168 80 232 116" style="animation-delay:2.99s"/>
      <path class="conn m m10 slow" d="M124 52 Q 192 95 232 148" style="animation-delay:3.02s"/>
      <path class="conn m m11 slow" d="M124 84 Q 162 55 232 20" style="animation-delay:3.05s"/>
      <path class="conn m m12 slow" d="M124 84 Q 198 68 232 52" style="animation-delay:3.08s"/>
      <path class="conn m m13 slow" d="M124 84 Q 174 84 232 84" style="animation-delay:3.11s"/>
      <path class="conn m m14 slow" d="M124 84 Q 186 98 232 116" style="animation-delay:3.14s"/>
      <path class="conn m m15 slow" d="M124 84 Q 160 112 232 148" style="animation-delay:3.17s"/>
      <path class="conn m m16 slow" d="M124 116 Q 200 72 232 20" style="animation-delay:3.20s"/>
      <path class="conn m m17 slow" d="M124 116 Q 176 85 232 52" style="animation-delay:3.23s"/>
      <path class="conn m m18 slow" d="M124 116 Q 184 98 232 84" style="animation-delay:3.26s"/>
      <path class="conn m m19 slow" d="M124 116 Q 158 116 232 116" style="animation-delay:3.29s"/>
      <path class="conn m m20 slow" d="M124 116 Q 202 130 232 148" style="animation-delay:3.32s"/>
      <path class="conn m m21 slow" d="M124 148 Q 176 88 232 20" style="animation-delay:3.35s"/>
      <path class="conn m m22 slow" d="M124 148 Q 184 102 232 52" style="animation-delay:3.38s"/>
      <path class="conn m m23 slow" d="M124 148 Q 158 114 232 84" style="animation-delay:3.41s"/>
      <path class="conn m m24 slow" d="M124 148 Q 200 130 232 116" style="animation-delay:3.44s"/>
      <path class="conn m m25 slow" d="M124 148 Q 170 148 232 148" style="animation-delay:3.47s"/>
    </g>
    <!-- nodes (rendered on top) -->
    <!-- left layer -->
    <g class="layer-left">
      <g style="--row-delay:0s" class="r1">
        <circle class="node r1" cx="16" cy="20" r="6"/>
      </g>
      <g style="--row-delay:0.15s" class="r2">
        <circle class="node r2" cx="16" cy="52" r="6"/>
      </g>
      <g style="--row-delay:0.30s" class="r3">
        <circle class="node r3" cx="16" cy="84" r="6"/>
      </g>
      <g style="--row-delay:0.45s" class="r4">
        <circle class="node r4" cx="16" cy="116" r="6"/>
      </g>
      <g style="--row-delay:0.60s" class="r5">
        <circle class="node r5" cx="16" cy="148" r="6"/>
      </g>
    </g>

    <!-- middle layer -->
    <g class="layer-middle">
      <g style="--row-delay:0s" class="r1">
        <circle class="node r1" cx="124" cy="20" r="6"/>
      </g>
      <g style="--row-delay:0.15s" class="r2">
        <circle class="node r2" cx="124" cy="52" r="6"/>
      </g>
      <g style="--row-delay:0.30s" class="r3">
        <circle class="node r3" cx="124" cy="84" r="6"/>
      </g>
      <g style="--row-delay:0.45s" class="r4">
        <circle class="node r4" cx="124" cy="116" r="6"/>
      </g>
      <g style="--row-delay:0.60s" class="r5">
        <circle class="node r5" cx="124" cy="148" r="6"/>
      </g>
    </g>

    <!-- right layer -->
    <g class="layer-right">
      <g style="--row-delay:0s" class="r1">
        <circle class="node r1" cx="232" cy="20" r="6"/>
      </g>
      <g style="--row-delay:0.15s" class="r2">
        <circle class="node r2" cx="232" cy="52" r="6"/>
      </g>
      <g style="--row-delay:0.30s" class="r3">
        <circle class="node r3" cx="232" cy="84" r="6"/>
      </g>
      <g style="--row-delay:0.45s" class="r4">
        <circle class="node r4" cx="232" cy="116" r="6"/>
      </g>
      <g style="--row-delay:0.60s" class="r5">
        <circle class="node r5" cx="232" cy="148" r="6"/>
      </g>
    </g>
  </g>
</svg>
"""
# Display the Neural Network SVG
display(HTML(svg_str_nn))

### Self-Play
Putting it all together. The agent plays against itself, generates data, and updates its brain.

In [8]:
svg_str_sp = """
<svg xmlns="http://www.w3.org/2000/svg" viewBox="40 100 240 130" width="100%" height="200" role="img" aria-hidden="true">
  <style>
    :root{
      --accent-svg: var(--accent, #0969da);
      --muted-svg: var(--text, #7a7a7aff);
      --self-play-svg: var(--accent, #0969da);
      --x-svg: var(--accent, #0969da);
      --o-svg: var(--antiaccent, #DA7A09);
    }
    
    /* AI agent (circle) moves between positions */
    .agent{fill:#fff;stroke:var(--muted-svg);stroke-width:2;opacity:0.8;}
    @keyframes moveAgent{
      0%{cx:40px}
      8%{cx:40px}
      16%{cx:220px}
      24%{cx:220px}
      32%{cx:40px}
      40%{cx:40px}
      48%{cx:220px}
      56%{cx:220px}
      64%{cx:40px}
      72%{cx:40px}
      80%{cx:220px}
      88%{cx:220px}
      96%{cx:40px}
      100%{cx:40px}
    }
    .agent{animation:moveAgent 15s ease-in-out infinite;}
    
    /* Game board in center */
    .board{fill:none;stroke:var(--muted-svg);stroke-width:1.2;opacity:0.7;}
    .board-cell{fill:none;stroke:var(--muted-svg);stroke-width:0.8;opacity:0.5;}
    
    /* Moves - appear alternately */
    .move{transform-box:fill-box;transform-origin:center;opacity:0;}
    @keyframes appear{from{opacity:0;transform:scale(0.3)}to{opacity:1;transform:scale(1)}}
    .move-x{stroke:var(--self-play-svg);stroke-width:2.4;stroke-linecap:round;fill:none;}
    .move-o{fill:none;stroke:var(--o-svg);stroke-width:2.4;}
    
    /* Staggered move appearances */
    .m1{animation:appear 400ms ease-out forwards;animation-delay:0.5s;}
    .m2{animation:appear 400ms ease-out forwards;animation-delay:3.6s;}
    .m3{animation:appear 400ms ease-out forwards;animation-delay:6.0s;}
    .m4{animation:appear 400ms ease-out forwards;animation-delay:9.0s;}
    .m5{animation:appear 400ms ease-out forwards;animation-delay:12.0s;}
    .m6{animation:appear 400ms ease-out forwards;animation-delay:15.0s;}
    
    /* Arrows showing turn flow */
    .arrow{fill:none;stroke:var(--muted-svg);stroke-width:1.8;opacity:0;stroke-dasharray:80;stroke-dashoffset:80;marker-end:url(#arrowhead);}
    @keyframes drawArrow{to{stroke-dashoffset:0;opacity:0.7}}
    @keyframes fadeArrow{to{opacity:0}}
    .a1{animation:drawArrow 400ms ease forwards, fadeArrow 300ms ease forwards;animation-delay:0.0s,0.6s;}
    .a2{animation:drawArrow 400ms ease forwards, fadeArrow 300ms ease forwards;animation-delay:2.4s,3.0s;}
    .a3{animation:drawArrow 400ms ease forwards, fadeArrow 300ms ease forwards;animation-delay:4.8s,5.4s;}
    .a4{animation:drawArrow 400ms ease forwards, fadeArrow 300ms ease forwards;animation-delay:7.2s,7.8s;}
    .a5{animation:drawArrow 400ms ease forwards, fadeArrow 300ms ease forwards;animation-delay:9.6s,10.2s;}
    .a6{animation:drawArrow 400ms ease forwards, fadeArrow 300ms ease forwards;animation-delay:12.0s,12.6s;}
  </style>
  
  <defs>
    <marker id="arrowhead" markerWidth="8" markerHeight="8" refX="7" refY="4" orient="auto">
      <path d="M1 1 L7 4 L1 7" fill="none" stroke="var(--muted-svg)" stroke-width="1.5" stroke-linejoin="round"/>
    </marker>
  </defs>
  
  <g transform="translate(30,40)">
    <!-- Single AI Agent that moves -->
    <circle class="agent" cy="120" r="24"/>
    
    <!-- Central Game Board (3x3 grid) -->
    <g transform="translate(108,90)">
      <rect class="board" x="0" y="0" width="44" height="44" rx="2"/>
      <line class="board-cell" x1="14.6" y1="0" x2="14.6" y2="44"/>
      <line class="board-cell" x1="29.3" y1="0" x2="29.3" y2="44"/>
      <line class="board-cell" x1="0" y1="14.6" x2="44" y2="14.6"/>
      <line class="board-cell" x1="0" y1="29.3" x2="44" y2="29.3"/>
      
      <!-- Moves on board (X and O) -->
      <!-- X moves (AI left) -->
      <g class="move m1 move-x">
        <line x1="4" y1="4" x2="10.6" y2="10.6"/>
        <line x1="10.6" y1="4" x2="4" y2="10.6"/>
      </g>
      <g class="move m3 move-x">
        <line x1="18.6" y1="18.6" x2="25.3" y2="25.3"/>
        <line x1="25.3" y1="18.6" x2="18.6" y2="25.3"/>
      </g>
      <g class="move m5 move-x">
        <line x1="33.4" y1="4" x2="40" y2="10.6"/>
        <line x1="40" y1="4" x2="33.4" y2="10.6"/>
      </g>
      
      <!-- O moves (AI right) -->
      <g class="move m2 move-o">
        <circle cx="37" cy="22" r="5.3"/>
      </g>
      <g class="move m4 move-o">
        <circle cx="7.3" cy="22" r="5.3"/>
      </g>
      <g class="move m6 move-o">
        <circle cx="22" cy="37" r="5.3"/>
      </g>
    </g>
    
    <!-- Turn arrows -->
    <path class="arrow a1" d="M64 120 L102 110"/>
    <path class="arrow a2" d="M196 120 L158 118"/>
    <path class="arrow a3" d="M64 120 L102 118"/>
    <path class="arrow a4" d="M196 120 L158 118"/>
    <path class="arrow a5" d="M64 120 L102 106"/>
    <path class="arrow a6" d="M196 120 L158 130"/>
  </g>
</svg>
"""
display(HTML(svg_str_sp))

## A Philosophical Note: Alien Intelligence

AlphaZero represents a shift in how we think about Artificial Intelligence.

Traditional chess engines were built on **human knowledge**. Grandmasters wrote rules: *"Knights are worth 3 points,"* *"Control the center."* The machine was a calculator executing human wisdom.

AlphaZero starts with **zero knowledge** (Tabula Rasa). It is given only the rules of the game. It derives its own strategy from first principles.
*   It doesn't play like a human. It plays like an alien that learned the game in isolation.
*   It makes moves that look "wrong" to human experts, only to be revealed as brilliant 20 moves later.
*   It proves that intuition (System 1) and reasoning (System 2) are not opposing forces, but multipliers of each other.

---

### Ready to Begin?

Start your journey with **[The Reasoner](./mcts.ipynb)**.